<a href="https://colab.research.google.com/github/keripikkaneboo/Hands-On-Machine-Learning-O-Reilly-/blob/main/13.%20Chapter13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bab 13: Loading and Preprocessing Data with TensorFlow

Saat bekerja dengan dataset yang sangat besar yang tidak muat dalam memori, kita memerlukan cara yang efisien untuk memuat dan memproses data. Bab ini memperkenalkan **Data API (`tf.data`)** dari TensorFlow, sebuah alat yang kuat untuk membangun pipeline input yang skalabel dan beperforma tinggi.

* **Data API (`tf.data`)**:
    * **Konsep Inti**: API ini berpusat pada objek `tf.data.Dataset`, yang merepresentasikan sebuah urutan item data.
    * **Pipeline Efisien**: Anda dapat membuat pipeline dengan merangkai (*chaining*) beberapa metode transformasi pada dataset, seperti:
        * `map()`: Menerapkan fungsi preprocessing pada setiap item.
        * `shuffle()`: Mengacak item data.
        * `batch()`: Mengelompokkan item ke dalam *batch*.
        * `prefetch()`: Memungkinkan data disiapkan di latar belakang saat model sedang training, mencegah GPU menunggu data dan meningkatkan utilisasi.
        * `cache()`: Menyimpan dataset dalam memori setelah pemrosesan pertama, mempercepat epoch-epoch berikutnya (berguna untuk dataset yang tidak terlalu besar).
    * **Membaca dari Berbagai Sumber**: Data API dapat membaca data dari file teks (CSV), file biner, dan format `TFRecord`, serta mendukung pembacaan dari banyak file secara paralel.

* **Format TFRecord**:
    * Format biner pilihan TensorFlow untuk menyimpan data dalam jumlah besar dan membacanya secara efisien.
    * TFRecord pada dasarnya adalah daftar rekaman biner, di mana setiap rekaman biasanya berisi **protocol buffer** yang diserialisasi.
    * **Protocol Buffer `Example`**: Protobuf standar yang digunakan di TensorFlow. Ia bertindak sebagai kamus (`map`) yang memetakan nama fitur ke nilainya (`BytesList`, `FloatList`, atau `Int64List`).

* **Lapisan Preprocessing Keras**:
    * Cara modern dan direkomendasikan untuk melakukan preprocessing adalah dengan memasukkannya sebagai lapisan di awal model Anda. Ini memiliki beberapa keuntungan:
        1.  Kode preprocessing hanya perlu ditulis sekali.
        2.  Model yang disimpan sudah mencakup preprocessing, sehingga mudah untuk dideploy dan mengurangi risiko *training/serving skew* (perbedaan antara preprocessing saat training dan saat inferensi).
    * **Lapisan yang Tersedia**:
        * **Encoding Fitur Kategorikal**:
            * `TextVectorization`: Mengubah teks menjadi urutan integer (indeks kata).
            * `Embedding`: Mengubah indeks integer menjadi vektor padat (*dense vector*) yang dapat dilatih. Ini adalah cara yang efisien untuk merepresentasikan kategori, terutama untuk kosa kata yang besar.
        * **Normalisasi Fitur**:
            * `Normalization`: Menstandardisasi fitur numerik (mengurangi rata-rata dan membagi dengan standar deviasi).
    * **Metode `adapt()`**: Lapisan preprocessing ini memiliki metode `adapt()` yang memungkinkan mereka mempelajari statistik yang diperlukan dari sampel data (misalnya, mempelajari kosa kata dari teks atau rata-rata dan standar deviasi dari fitur numerik).

* **TensorFlow Datasets (TFDS)**:
    * Sebuah library yang menyediakan akses mudah ke ratusan dataset umum.
    * Fungsi `tfds.load()` mengunduh data dan mengembalikannya sebagai `tf.data.Dataset`, siap untuk diproses lebih lanjut.

### 1. Membangun Pipeline Input dengan Data API
Contoh dasar membuat pipeline `tf.data`.

```python
import tensorflow as tf

# Membuat dataset dari tensor di memori
dataset = tf.data.Dataset.from_tensor_slices(tf.range(10))
print("Dataset awal:", list(dataset.as_numpy_iterator()))

# Merangkai transformasi
dataset = dataset.repeat(3).batch(7, drop_remainder=True)
print("\nDataset setelah repeat(3) dan batch(7):")
for item in dataset:
    print(item)

# Transformasi lebih lanjut dengan map, shuffle, dan prefetch
dataset = dataset.map(lambda x: x * 2) # Mengalikan setiap elemen dengan 2
dataset = dataset.unbatch() # Mengembalikan ke elemen individual
dataset = dataset.shuffle(buffer_size=10).batch(5).prefetch(1)

print("\nPipeline akhir:")
for item in dataset:
    print(item)
```

### 2. Menulis dan Membaca File TFRecord
Contoh ini menunjukkan cara membuat file `.tfrecord` dari data, lalu membacanya kembali.

```python
# Menulis ke file TFRecord
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
    f.write(b"Ini adalah rekaman pertama.")
    f.write(b"Dan ini yang kedua.")

# Membaca dari file TFRecord
filepaths = ["my_data.tfrecord"]
dataset = tf.data.TFRecordDataset(filepaths)
for item in dataset:
    print(item)

# --- Contoh dengan `Example` Protobuf ---
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Feature, Features, Example

# Membuat `Example` protobuf
person_example = Example(
    features=Features(
        feature={
            "name": Feature(bytes_list=BytesList(value=[b"Alice"])),
            "id": Feature(int64_list=Int64List(value=[123])),
            "emails": Feature(bytes_list=BytesList(value=[b"a@b.com", b"c@d.com"]))
        }))

# Menulis Example yang diserialisasi ke file TFRecord
with tf.io.TFRecordWriter("my_contacts.tfrecord") as f:
    f.write(person_example.SerializeToString())

# Membaca dan mem-parsing Example
feature_description = {
    "name": tf.io.FixedLenFeature([], tf.string, default_value=""),
    "id": tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "emails": tf.io.VarLenFeature(tf.string),
}

dataset = tf.data.TFRecordDataset(["my_contacts.tfrecord"])
for serialized_example in dataset:
    parsed_example = tf.io.parse_single_example(serialized_example,
                                                 feature_description)
    print("\nParsed Example:", parsed_example)
```

### 3. Menggunakan Lapisan Preprocessing Keras
Contoh bagaimana lapisan `TextVectorization` dan `Embedding` digunakan dalam sebuah model.

```python
from tensorflow import keras
import numpy as np

# Contoh data teks
some_texts = np.array(["I love deep learning", "deep learning is awesome"])

# 1. Buat dan adaptasi TextVectorization layer
# max_tokens: ukuran kosa kata, output_sequence_length: panjang urutan output
text_vec_layer = keras.layers.TextVectorization(max_tokens=100, output_sequence_length=5)
text_vec_layer.adapt(some_texts)

print("\nKosa kata:", text_vec_layer.get_vocabulary())
print("Hasil vectorization:", text_vec_layer(some_texts))


# 2. Menggunakan Embedding dalam sebuah model
embedding_dim = 2
vocab_size = text_vec_layer.vocabulary_size()

model = keras.models.Sequential([
    text_vec_layer, # Lapisan preprocessing
    keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)
])

# Menjalankan model untuk melihat hasil embedding
embedding_output = model.predict(some_texts)
print("\nOutput embedding:\n", embedding_output)
```

### 4. Menggunakan TensorFlow Datasets (TFDS)
Cara termudah untuk mendapatkan akses ke ratusan dataset standar.

```python
# Diperlukan untuk Google Colab
!pip install -q -U tensorflow_datasets

import tensorflow_datasets as tfds

# Memuat dataset imdb_reviews, sudah dibagi dan dibatch
# as_supervised=True akan mengembalikan tuple (input, label)
dataset, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_set = dataset["train"].batch(32).prefetch(1)

print("\nInfo dataset IMDb:", info.splits)

# Dataset ini sekarang siap untuk dimasukkan ke model Keras
# for X_batch, y_batch in train_set.take(1):
#     print(X_batch.shape, y_batch.shape)
```
